Part 02

Environment

In [28]:
import numpy as np
import random

class Environment:
    def __init__(self, grid_size):
        self.n_row = grid_size
        self.n_col = grid_size
        self.n_state = self.n_row * self.n_col
        self.n_action = 4

        # Define actions
        self.action = [[0, -1], [1, 0], [0, 1], [-1, 0]]
        self.action_text = ['←', '↓', '→', '↑']

        # Define initial state
        self.state_0 = 3 * self.n_row + 3  # Center of the grid (3, 3)

        # Define terminal states
        self.terminal_states = {6 * self.n_row + 0: -1, 0 * self.n_row + 6: 1}

        # Define model
        self.model = [[[] for _ in range(self.n_action)] for _ in range(self.n_state)]
        for s in range(self.n_state):
            for a in range(self.n_action):
                row, col = np.divmod(s, self.n_row)
                act = self.action[a]
                row_, col_ = row + act[0], col + act[1]
                state_ = row_ * self.n_row + col_
                outsidecheck = (row_ < 0) or (col_ < 0) or (row_ >= self.n_row) or (col_ >= self.n_col)

                if outsidecheck:
                    self.model[s][a].append([1.0, s, 0.0, False])  # if trying to move outside the grid
                elif state_ in self.terminal_states:
                    self.model[s][a].append([1.0, state_, self.terminal_states[state_], True])  # if moving to a terminal state
                else:
                    self.model[s][a].append([1.0, state_, 0.0, False])  # normal movement

# Function to encode the state using one-hot encoding
def Affine(state, env):
    fv = np.zeros(env.n_state)
    fv[state] = 1
    return fv




In [29]:
# Function to compute the exact value function
def compute_exact_value_function(env, policy, gamma):
    P = np.zeros((env.n_state, env.n_state))
    R = np.zeros(env.n_state)

    for s in range(env.n_state):
        if s in env.terminal_states:
            continue
        for a in range(env.n_action):
            if policy[s][a] > 0:  # Only consider actions with non-zero probability
                for p, s_, r, t in env.model[s][a]:
                    P[s, s_] += policy[s][a] * p
                    R[s] += policy[s][a] * p * r

    I = np.eye(env.n_state)
    V = np.linalg.solve(I - gamma * P, R)

    # Set terminal states to zero
    for state in env.terminal_states:
        V[state] = 0

    return np.round(V, 2)

In [30]:
# Gradient Monte Carlo method
def Gradient_Monte_Carlo(max_ep, alpha, gamma, policy, env):
    w = np.zeros(env.n_state)

    for ep in range(max_ep):
        s = env.state_0
        Traces = []

        # Generate an episode
        while True:
            a = np.random.choice(np.arange(env.n_action), p=policy[s])
            _, s_, r, t = env.model[s][a][0]

            Traces.append([s, a, r])
            s = s_

            if t:
                break

        G = 0
        for t, trace in enumerate(Traces[::-1]):
            G = gamma * G + trace[2]
            x = Affine(trace[0], env)
            if trace[0] not in env.terminal_states:
                w += alpha * (G - np.dot(w, x)) * x

    return np.round(w, 2)

In [31]:
# Semi-Gradient TD(0) method
def Semi_Gradient_TD0(max_ep, alpha, gamma, policy, env):
    w = np.zeros(env.n_state)

    for ep in range(max_ep):
        s = env.state_0

        while True:
            a = np.random.choice(np.arange(env.n_action), p=policy[s])
            _, s_, r, t = env.model[s][a][0]

            x = Affine(s, env)
            x_ = Affine(s_, env)
            if s not in env.terminal_states:
                w += alpha * (r + gamma * np.dot(w, x_) - np.dot(w, x)) * x

            s = s_

            if t:
                break

    return np.round(w, 2)

In [32]:
# Function to find the optimal policy based on the value function
def find_policy(gamma, V, env):
    policy_opt = np.zeros((env.n_state, env.n_action))
    for s in range(env.n_state):
        q_values = []
        for a in range(env.n_action):
            q_value = 0
            for p, s_, r, t in env.model[s][a]:
                q_value += p * (r + gamma * np.dot(V, Affine(s_, env)))
            q_values.append(q_value)
        best_action = np.argmax(q_values)
        policy_opt[s] = np.eye(env.n_action)[best_action]
    return policy_opt

In [51]:
# Define and initialize the environment
env = Environment(grid_size=7)

# Define the initial random policy
policy = np.ones((env.n_state, env.n_action)) / env.n_action

# Compute the exact value function
gamma = 0.9
V_exact = compute_exact_value_function(env, policy, gamma)

# Perform Gradient Monte Carlo
alpha = 0.01
max_ep = 10000
VF_gmc = Gradient_Monte_Carlo(max_ep, alpha, gamma, policy, env)

# Perform Semi-Gradient TD(0)
VF_td0 = Semi_Gradient_TD0(max_ep, alpha, gamma, policy, env)

# Reshape the value functions to match the grid layout
V_exact_grid = V_exact.reshape((env.n_row, env.n_col))
VF_gmc_grid = VF_gmc.reshape((env.n_row, env.n_col))
VF_td0_grid = VF_td0.reshape((env.n_row, env.n_col))

# Function to determine the optimal policy based on the value function
def get_optimal_policy_from_value(V_grid, env):
    optimal_policy = np.full((env.n_row, env.n_col), 'x', dtype='<U2')
    for i in range(env.n_row):
        for j in range(env.n_col):
            state = i * env.n_row + j
            if state not in env.terminal_states:
                q_values = []
                for a in range(env.n_action):
                    q_value = 0
                    for p, s_, r, t in env.model[state][a]:
                        q_value += p * (r + gamma * V_grid[s_ // env.n_row, s_ % env.n_col])
                    q_values.append(q_value)
                best_action = np.argmax(q_values)
                optimal_policy[i, j] = env.action_text[best_action]
    return optimal_policy

optimal_policy_exact = get_optimal_policy_from_value(V_exact_grid, env)
optimal_policy_gmc = get_optimal_policy_from_value(VF_gmc_grid, env)
optimal_policy_td0 = get_optimal_policy_from_value(VF_td0_grid, env)



In [52]:
# Print the value functions and optimal policies
def print_value_function_and_policy(V_grid, policy, title):
    print(f"{title} Value Function:")
    for row in V_grid:
        print(" ".join(f"{val:5.2f}" for val in row))
    print("\nOptimal Policy:")
    for row in policy:
        print(" ".join(row))
    print("\n")

print_value_function_and_policy(V_exact_grid, optimal_policy_exact, "Exact")
print_value_function_and_policy(VF_gmc_grid, optimal_policy_gmc, "Gradient Monte Carlo")
print_value_function_and_policy(VF_td0_grid, optimal_policy_td0, "Semi-Gradient TD(0)")

Exact Value Function:
 0.00  0.01  0.03  0.08  0.20  0.46  0.00
-0.01  0.00  0.02  0.06  0.13  0.27  0.46
-0.03 -0.02  0.00  0.03  0.07  0.13  0.20
-0.08 -0.06 -0.03  0.00  0.03  0.06  0.08
-0.20 -0.13 -0.07 -0.03  0.00  0.02  0.03
-0.46 -0.27 -0.13 -0.06 -0.02  0.00  0.01
 0.00 -0.46 -0.20 -0.08 -0.03 -0.01  0.00

Optimal Policy:
→ → → → → → x
→ → → → → → ↑
↑ → → → → ↑ ↑
↑ ↑ → → ↑ ↑ ↑
↑ ↑ → → ↑ ↑ ↑
↑ → → → → ↑ ↑
x → → → → → ↑


Gradient Monte Carlo Value Function:
-0.00  0.01  0.05  0.12  0.24  0.52  0.00
-0.01  0.01  0.03  0.09  0.19  0.33  0.46
-0.04 -0.03  0.00  0.06  0.09  0.16  0.17
-0.10 -0.07 -0.02  0.00  0.02  0.07  0.07
-0.23 -0.15 -0.06 -0.04 -0.01  0.04  0.02
-0.45 -0.28 -0.15 -0.10 -0.06 -0.00 -0.01
 0.00 -0.46 -0.20 -0.10 -0.06 -0.03  0.00

Optimal Policy:
→ → → → → → x
→ → → → → ↑ ↑
↑ ↑ → → ↑ ↑ ↑
↑ → → ↑ ↑ ↑ ↑
↑ → ↑ ↑ → ↑ ↑
↑ → ↑ ↑ → ↑ ↑
x → → → → → ↓


Semi-Gradient TD(0) Value Function:
 0.00  0.01  0.03  0.07  0.20  0.50  0.00
-0.01  0.00  0.02  0.06  0.12  0.26  0.47